# Image Binarization

The dataset consist of historical images, the class are:
- Class 1: background
- Class 2: foreground

This notebook is the exactly google colab notebook I have used to train and test the model. All the images are taken from my google drive. The images used to test the approaches are in the directory "Test_images_approach_"



# 0. Dataset and Basic Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# code from: https://colab.research.google.com/github/VidushiBhatia/U-Net-Implementation/blob/main/U_Net_for_Image_Segmentation_From_Scratch_Using_TensorFlow_v4.ipynb#scrollTo=dwjr7eZQEK36

# for data load
import os

# for reading and processing images
import imageio
from PIL import Image
import cv2

# for visualizations
import matplotlib.pyplot as plt

import numpy as np # for using np arrays

# for bulding and running deep learning model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks, metrics



import pandas as pd

MODEL_NAME = 'U-Net'
os.makedirs(f'/content/drive/MyDrive/models/{MODEL_NAME}', exist_ok = True)
N_CLASSES = 2

MC_BEST = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5', save_best_only=True, save_weights_only=True, mode='max', save_freq="epoch", verbose=1)
LOG = callbacks.CSVLogger(f'/content/drive/MyDrive/models/{MODEL_NAME}/{MODEL_NAME}.log', append=True)


# 1. Helper Function for Data Processing


## 1.1  Load  and Process Data

In [ ]:
# code from: https://colab.research.google.com/github/VidushiBhatia/U-Net-Implementation/blob/main/U_Net_for_Image_Segmentation_From_Scratch_Using_TensorFlow_v4.ipynb#scrollTo=dwjr7eZQEK36

def LoadData (path1, path2):
    """
    Looks for relevant filenames in the shared path
    Returns 2 lists for original and masked files respectively
    
    """
    # Read the images folder like a list
    image_dataset = os.listdir(path1)
    mask_dataset = os.listdir(path2)

    # Make a list for images and masks filenames
    orig_img = []
    mask_img = []
    for file in image_dataset:
        orig_img.append(file)
    for file in mask_dataset:
        mask_img.append(file)

    # Sort the lists to get both of them in same order (the dataset has exactly the same name for images and corresponding masks)
    orig_img.sort()
    mask_img.sort()
    
    return orig_img, mask_img
    



In [ ]:
def pad_image_to_tile_multiple(image3, tile_size, padding="CONSTANT"):
    imagesize = image3.shape
    if (imagesize[0] % tile_size[0]) != 0 or (imagesize[1] % tile_size[1]) != 0: #it is not of the correct size 
  
      target_height = imagesize[0] - (imagesize[0] % tile_size[0]) + tile_size[0]
      target_width = imagesize[1] - (imagesize[1] % tile_size[1]) + tile_size[1]

      add_height = target_height - imagesize[0]
      add_width = target_width - imagesize[1]

      ret = np.pad(image3, ((0, add_height) , (0, add_width)), "constant", constant_values=0)
    else: #if it is already of the correct size 
      ret = image3

    return ret

def reverse_padding(original_image, changed_image):
  original_size = original_image.shape
  return changed_image[0:original_size[0], 0:original_size[1]]
    
# code adapted from: https://stackoverflow.com/questions/38235643/getting-started-with-tensorflow-split-image-into-sub-images

def split_image(image3, tile_size):
    image_shape = tf.shape(image3)
    tile_rows = tf.reshape(image3, [image_shape[0], -1, tile_size[1]])
    serial_tiles = tf.transpose(tile_rows, [1, 0, 2])
    return tf.reshape(serial_tiles, [-1, tile_size[1], tile_size[0]])

def unsplit_image(tiles4, image_shape):
    tile_width = tf.shape(tiles4)[1]
    serialized_tiles = tf.reshape(tiles4, [-1, image_shape[0], tile_width])
    rowwise_tiles = tf.transpose(serialized_tiles, [1, 0, 2])
    return tf.reshape(rowwise_tiles, [image_shape[0], image_shape[1]])

In [ ]:
import numpy as np

def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [ ]:
'''
This function get the images and the training mask, and fro every every image, mask create
a block of size [target_shape_img], and add this block to the training array X

'''
def PreprocessData(img, mask, target_shape_block, path1, path2, change_color):
  '''
  Processes the images and mask present in the shared list and path, split the images
  and the masks in the target_shape_block indicated, and add the block to the output list

  target_shape_block is a 2 dimensional array, with the expected blok size
  '''

  X = {}
  y = {}
  shape_images_dict = {}
  for file in img:
    index = img.index(file)
    single_mask_ind = mask[index]

    #get images and mask
    single_image = imageio.imread(os.path.join(path1, file))
    if change_color:
      single_image = cv2.cvtColor(single_image, cv2.COLOR_BGR2GRAY)
    single_image = single_image / 255

    single_mask = imageio.imread(os.path.join(path2, single_mask_ind))
    if change_color:
      single_mask = cv2.cvtColor(single_mask, cv2.COLOR_BGR2GRAY)
    single_mask = single_mask / 255

    #split the images, both images and mash have the same size
    
    single_image = pad_image_to_tile_multiple(single_image, target_shape_block)
    single_mask = pad_image_to_tile_multiple(single_mask, target_shape_block)

    single_image=tf.convert_to_tensor(single_image,dtype=tf.float32)# this is converting the the numpy array further
    single_mask = tf.convert_to_tensor(single_mask, dtype=tf.float32)

    tiles_image = split_image(single_image, target_shape_block)
    tiles_mask = split_image(single_mask, target_shape_block)

    shape_images_dict[file] = single_image.shape

    for tile in range(len(tiles_image)):
      X[file.replace(".png", f"_{tile}")] = tiles_image[tile]
      y[file.replace(".png", f"_{tile}")] = tiles_mask[tile]

  #create a proper dataset
  m = len(X.keys())
  i_h = target_shape_block[0]
  i_w = target_shape_block[1]
  X_ = np.zeros((m, i_h, i_w), dtype=np.float)
  y_ = np.zeros((m, i_h, i_w), dtype=np.int32)

  for i in range(m):
    X_[i] = X[list(X.keys())[i]]
    y_[i] = y[list(y.keys())[i]]

  return X_, y_, shape_images_dict #return also the size of the images after the padding

In [ ]:
# Results of Validation Dataset
def VisualizeResults(index):
    img = X_valid[index]
    img = img[np.newaxis, ...]
    pred_y = unet.predict(img)
    pred_mask = tf.argmax(pred_y[0], axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    fig, arr = plt.subplots(1, 3, figsize=(15, 15))
    arr[0].imshow(X_valid[index])
    arr[0].set_title('Processed Image')
    arr[1].imshow(y_valid[index])
    arr[1].set_title('Actual Masked Image ')
    arr[2].imshow(pred_mask[:,:,0])
    arr[2].set_title('Predicted Masked Image ')
    

##1.2 Constructing the U-Net Architecture

In [ ]:
# code from: https://colab.research.google.com/github/VidushiBhatia/U-Net-Implementation/blob/main/U_Net_for_Image_Segmentation_From_Scratch_Using_TensorFlow_v4.ipynb#scrollTo=2hCcs14OEK4C


#U-Net Encoder Block
def EncoderMiniBlock(inputs, n_filters=32, dropout_prob=0.3, max_pooling=True):
    """
    This block uses multiple convolution layers, max pool, relu activation to create an architecture for learning. 
    Dropout can be added for regularization to prevent overfitting. 
    The block returns the activation values for next layer along with a skip connection which will be used in the decoder
    """
    # Add 2 Conv Layers with relu activation and HeNormal initialization using TensorFlow 
    # Proper initialization prevents from the problem of exploding and vanishing gradients 
    # 'Same' padding will pad the input to conv layer such that the output has the same height and width (hence, is not reduced in size) 
    conv = Conv2D(n_filters, 
                  3,   # Kernel size   
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = Conv2D(n_filters, 
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
    
    # Batch Normalization will normalize the output of the last layer based on the batch's mean and standard deviation
    conv = BatchNormalization()(conv, training=False)

    # In case of overfitting, dropout will regularize the loss and gradient computation to shrink the influence of weights on output
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    # Pooling reduces the size of the image while keeping the number of channels same
    # Pooling has been kept as optional as the last encoder layer does not use pooling (hence, makes the encoder block flexible to use)
    # Below, Max pooling considers the maximum of the input slice for output computation and uses stride of 2 to traverse across input image
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)    
    else:
        next_layer = conv

    # skip connection (without max pooling) will be input to the decoder layer to prevent information loss during transpose convolutions      
    skip_connection = conv
    
    return next_layer, skip_connection

# U-Net Decoder Block
def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=32):
    """
    Decoder Block first uses transpose convolution to upscale the image to a bigger size and then,
    merges the result with skip layer results from encoder block
    Adding 2 convolutions with 'same' padding helps further increase the depth of the network for better predictions
    The function returns the decoded layer output
    """
    # Start with a transpose convolution layer to first increase the size of the image
    up = Conv2DTranspose(
                 n_filters,
                 (3,3),    # Kernel size
                 strides=(2,2),
                 padding='same')(prev_layer_input)

    # Merge the skip connection from previous block to prevent information loss
    merge = concatenate([up, skip_layer_input], axis=3)
    
    # Add 2 Conv Layers with relu activation and HeNormal initialization for further processing
    # The parameters for the function are similar to encoder
    conv = Conv2D(n_filters, 
                 3,     # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(merge)
    conv = Conv2D(n_filters,
                 3,   # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(conv)
    return conv

# Compile U-Net Blocks
def UNetCompiled(input_size=(128, 128, 3), n_filters=32, n_classes=3):
    """
    Combine both encoder and decoder blocks according to the U-Net research paper
    Return the model as output 
    """
    # Input size represent the size of 1 image (the size used for pre-processing) 
    inputs = Input(input_size)
    
    # Encoder includes multiple convolutional mini blocks with different maxpooling, dropout and filter parameters
    # Observe that the filters are increasing as we go deeper into the network which will increasse the # channels of the image 
    cblock1 = EncoderMiniBlock(inputs, n_filters,dropout_prob=0, max_pooling=True)
    cblock2 = EncoderMiniBlock(cblock1[0],n_filters*2,dropout_prob=0, max_pooling=True)
    cblock3 = EncoderMiniBlock(cblock2[0], n_filters*4,dropout_prob=0, max_pooling=True)
    cblock4 = EncoderMiniBlock(cblock3[0], n_filters*8,dropout_prob=0.3, max_pooling=True)
    cblock5 = EncoderMiniBlock(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False) 
    
    # Decoder includes multiple mini blocks with decreasing number of filters
    # Observe the skip connections from the encoder are given as input to the decoder
    # Recall the 2nd output of encoder block was skip connection, hence cblockn[1] is used
    ublock6 = DecoderMiniBlock(cblock5[0], cblock4[1],  n_filters * 8)
    ublock7 = DecoderMiniBlock(ublock6, cblock3[1],  n_filters * 4)
    ublock8 = DecoderMiniBlock(ublock7, cblock2[1],  n_filters * 2)
    ublock9 = DecoderMiniBlock(ublock8, cblock1[1],  n_filters)

    # Complete the model with 1 3x3 convolution layer (Same as the prev Conv Layers)
    # Followed by a 1x1 Conv layer to get the image to the desired size. 
    # Observe the number of channels will be equal to number of output classes
    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(n_classes, 1, padding='same')(conv9)
    
    # Define the model
    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model

# 2. Approach 1: Concatenation Features

## 2.1 Load and View Data

In [ ]:
""" Load Train Set and view some examples """
# Call the apt function
path1 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_1/org/'
path2 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_1/gt/'
img, mask = LoadData (path1, path2)

# View an example of image and corresponding mask 
show_images = 4
for i in range(show_images):
    img_view  = imageio.imread(path1 + img[i])
    img_view = cv2.cvtColor(img_view, cv2.COLOR_BGR2GRAY)
    mask_view = imageio.imread(path2 + mask[i])
    mask_view = cv2.cvtColor(mask_view, cv2.COLOR_BGR2GRAY)

    print(img_view.shape)
    print(mask_view.shape)
    fig, arr = plt.subplots(1, 2, figsize=(15, 15))
    arr[0].imshow(img_view)
    arr[0].set_title('Image '+ str(i))
    arr[1].imshow(mask_view)
    arr[1].set_title('Masked Image '+ str(i))

## 2.2 Preprocessing Data

In [ ]:
# Define the desired shape
target_shape_img_blok = [128, 128]

# Process data using apt helper function
X, y, _ = PreprocessData(img, mask, target_shape_img_blok, path1, path2, True)

# QC the shape of output and classes in output dataset 
print("X Shape:", X.shape)
print("Y shape:", y.shape)
# There are 3 classes : background, pet, outline
print(np.unique(y))

# Visualize the output
image_index = 2
fig, arr = plt.subplots(1, 2, figsize=(15, 15))
arr[0].imshow(X[image_index])
arr[0].set_title('Processed Image')
arr[1].imshow(y[image_index])
arr[1].set_title('Processed Masked Image ')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=123)


## 2.3 Build the mdoel



In [ ]:
# Call the helper function for defining the layers for the model, given the input image size
unet = UNetCompiled(input_size=(128,128,1), n_filters=32, n_classes=2)

# Check the summary to better interpret how the output dimensions change in each layer
#unet.summary()

## 2.4 Compile and Run the model

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Run the model in a mini-batch fashion and compute the progress for each epoch
batch_size= 32
epochs = 15
results = unet.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=[MC_BEST, LOG])
hist =results.history

## 2.5 Evaluate Model Result

In [ ]:
'''
unet = UNetCompiled(input_size=(128,128,1), n_filters=32, n_classes=2)
unet.load_weights(f'/content/drive/MyDrive/models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5')
unet.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
unet.load_weights(f'/content/drive/MyDrive/models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5')

#load history
hist = pd.read_csv(f'/content/drive/MyDrive/models/{MODEL_NAME}/{MODEL_NAME}.log', sep=",", engine="python")
'''

In [ ]:
# High Bias is a characteristic of an underfitted model and we would observe low accuracies for both train and validation set
# High Variance is a characterisitic of an overfitted model and we would observe high accuracy for train set and low for validation set
# To check for bias and variance plit the graphs for accuracy 
# I have plotted for loss too, this helps in confirming if the loss is decreasing with each iteration - hence, the model is optimizing fine
fig, axis = plt.subplots(1, 2, figsize=(20, 5))
axis[0].plot(hist["loss"], color='r', label = 'train loss')
axis[0].plot(hist["val_loss"], color='b', label = 'dev loss')
axis[0].set_title('Loss Comparison')
axis[0].legend()
axis[1].plot(hist["accuracy"], color='r', label = 'train accuracy')
axis[1].plot(hist["val_accuracy"], color='b', label = 'dev accuracy')
axis[1].set_title('Accuracy Comparison')
axis[1].legend()

# RESULTS
# The train loss is consistently decreasing showing that Adam is able to optimize the model and find the minima
# The accuracy of train and validation is ~90% which is high enough, so low bias
# and the %s aren't that far apart, hence low variance

## 2.6 View Predicted Segmentations and Test on validation images

In [ ]:
unet.evaluate(X_valid, y_valid)

In [ ]:
# Add any index to contrast the predicted mask with actual mask
index = 8
VisualizeResults(index)

Test on different Images

In [ ]:
path1 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_1/val_images/org/'
path2 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_1/val_images/gt/'
img, mask = LoadData (path1, path2)

# Define the desired shape
target_shape_img_blok = [128, 128]

# Process data using apt helper function
X, y, shape_images_dict = PreprocessData(img, mask, target_shape_img_blok, path1, path2, False)

# QC the shape of output and classes in output dataset 
print("X Shape:", X.shape)
print("Y shape:", y.shape)
# There are 3 classes : background, pet, outline
print(np.unique(y))

# Visualize the output
image_index = 9
fig, arr = plt.subplots(1, 2, figsize=(15, 15))
arr[0].imshow(X[image_index])
arr[0].set_title('Processed Image')
arr[1].imshow(y[image_index])
arr[1].set_title('Processed Masked Image ')

In [ ]:
# doing the prediction
img = X[0]
img = img[np.newaxis, ...]
pred_y = unet.predict(img)
pred_mask_test = tf.argmax(pred_y[0], axis=-1)
pred_mask_test = pred_mask_test[..., tf.newaxis]

first = True
#for each box do the prediction and get the results
for i in range(1,X.shape[0]):
# doing the prediction
  img = X[i]
  img = img[np.newaxis, ...]
  pred_y = unet.predict(img)
  pred_mask = tf.argmax(pred_y[0], axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  
  if first:
    blocks = tf.stack((pred_mask_test[:,:,0], pred_mask[:,:,0]), axis=0)
    gt_blocks = tf.stack((y[0], y[i]), axis=0)
    first = False
  else:
    add = tf.concat(pred_mask[:,:,0], axis=1)
    add = tf.reshape(add, [1,target_shape_img_blok[0], target_shape_img_blok[1]])
    blocks = tf.concat((blocks, add), axis=0)

    add = tf.concat(y[i], axis=1)
    add = tf.reshape(add, [1,target_shape_img_blok[0], target_shape_img_blok[1]])
    gt_blocks = tf.concat((gt_blocks, add), axis=0)

In [ ]:
shape_images_dict.keys()

In [ ]:
#create the images
shape_block = target_shape_img_blok
image_shape_pad = [shape_images_dict["z802.png"], shape_images_dict["z822.png"]]
Test_images_approach_1 = "/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_1"

#create a directory with the name of the validation images
dir_val_img = ["z802", "z822"]
file_name = "F1s.png" #name of the file im going to take to reverse the passing


b = 0
for i in range(2): #for each validation images
  img_shape = image_shape_pad[i]
  n_blocks_for_image = round((img_shape[0] * img_shape[1]) / (shape_block[0] * shape_block[1]))
  

  # reconstruct the predicted images, and save thos in the apposit directory
  tiles = tf.slice(blocks, begin=[b, 0, 0], size=[n_blocks_for_image, shape_block[0], shape_block[1]])
  img = unsplit_image(tiles, img_shape)

  #Crete the GT image
  gt_tiles = tf.slice(gt_blocks, begin=[b, 0, 0], size=[n_blocks_for_image, shape_block[0], shape_block[1]])
  gt_img = unsplit_image(gt_tiles, img_shape)

  b += n_blocks_for_image

  #remove the padding before saving the images

  #get the image without padding
  img_no_pad = imageio.imread(os.path.join(Test_images_approach_1, "val_images", dir_val_img[i], file_name))
  img = reverse_padding(img_no_pad, img)
  gt_img = reverse_padding(img_no_pad, gt_img)

  #save the image
  plt.imsave(f"{Test_images_approach_1}/output/gt/FN{i}_GT.png", gt_img)
  plt.imsave(f"{Test_images_approach_1}/output/images/FN{i}.png", img)




# 3. Approach 2: Executing above Functions to Train the Model



## 3.1 Load and View Data

In [ ]:
""" Load Train Set and view some examples """
# Call the apt function
path1 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_2/org/'
path2 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_2/gt/'
img, mask = LoadData (path1, path2)

# View an example of image and corresponding mask 
show_images = 5
for i in range(show_images):
    img_view  = imageio.imread(path1 + img[i])
    mask_view = imageio.imread(path2 + mask[i])
    print(img_view.shape)
    print(mask_view.shape)
    fig, arr = plt.subplots(1, 2, figsize=(15, 15))
    arr[0].imshow(img_view)
    arr[0].set_title('Image '+ str(i))
    arr[1].imshow(mask_view)
    arr[1].set_title('Masked Image '+ str(i))

## 3.2 Process Data

In [ ]:
# Define the desired shape
target_shape_img_blok = [128, 128]

# Process data using apt helper function
X, y, image_shape_pad = PreprocessData(img, mask, target_shape_img_blok, path1, path2, False)

# QC the shape of output and classes in output dataset 
print("X Shape:", X.shape)
print("Y shape:", y.shape)
# There are 3 classes : background, pet, outline
print(np.unique(y))

# Visualize the output
image_index = 0
fig, arr = plt.subplots(1, 2, figsize=(15, 15))
arr[0].imshow(X[image_index])
arr[0].set_title('Processed Image')
arr[1].imshow(y[image_index])
arr[1].set_title('Processed Masked Image ')

In [ ]:
#Use scikit-learn's function to split the dataset
# Here, I have used 20% data as test/valid set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=123)

## 3.3 Buld U-Net Architecture

In [ ]:
# Call the helper function for defining the layers for the model, given the input image size
unet = UNetCompiled(input_size=(128,128,1), n_filters=32, n_classes=2)

# Check the summary to better interpret how the output dimensions change in each layer
#unet.summary()

## 3.4 Compile and Run Model

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Run the model in a mini-batch fashion and compute the progress for each epoch
batch_size= 32
epochs = 10
results = unet.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=[MC_BEST, LOG])
hist =results.history

## 3.5 Evaluate Model Result

In [ ]:
'''
unet = UNetCompiled(input_size=(128,128,1), n_filters=32, n_classes=2)
unet.load_weights(f'/content/drive/MyDrive/models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5')
unet.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
unet.load_weights(f'/content/drive/MyDrive/models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5')

#load history
hist = pd.read_csv(f'/content/drive/MyDrive/models/{MODEL_NAME}/{MODEL_NAME}.log', sep=",", engine="python")
'''

In [ ]:
# High Bias is a characteristic of an underfitted model and we would observe low accuracies for both train and validation set
# High Variance is a characterisitic of an overfitted model and we would observe high accuracy for train set and low for validation set
# To check for bias and variance plit the graphs for accuracy 
# I have plotted for loss too, this helps in confirming if the loss is decreasing with each iteration - hence, the model is optimizing fine
fig, axis = plt.subplots(1, 2, figsize=(20, 5))
axis[0].plot(hist["loss"], color='r', label = 'train loss')
axis[0].plot(hist["val_loss"], color='b', label = 'dev loss')
axis[0].set_title('Loss Comparison')
axis[0].legend()
axis[1].plot(hist["accuracy"], color='r', label = 'train accuracy')
axis[1].plot(hist["val_accuracy"], color='b', label = 'dev accuracy')
axis[1].set_title('Accuracy Comparison')
axis[1].legend()

# RESULTS
# The train loss is consistently decreasing showing that Adam is able to optimize the model and find the minima
# The accuracy of train and validation is ~90% which is high enough, so low bias
# and the %s aren't that far apart, hence low variance

## 3.6 View Predicted Segmentations and Test on validation images

In [ ]:
unet.evaluate(X_valid, y_valid)

In [ ]:
# Add any index to contrast the predicted mask with actual mask
index = 7
VisualizeResults(index)

Test on binarized different images

In [ ]:
path1 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_2/val_images/org/'
path2 = '/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_2/val_images/gt/'
img, mask = LoadData (path1, path2)

# Define the desired shape
target_shape_img_blok = [128, 128]

# Process data using apt helper function
X, y, shape_images_dict = PreprocessData(img, mask, target_shape_img_blok, path1, path2, False)

# QC the shape of output and classes in output dataset 
print("X Shape:", X.shape)
print("Y shape:", y.shape)
# There are 3 classes : background, pet, outline
print(np.unique(y))

# Visualize the output
image_index = 9
fig, arr = plt.subplots(1, 2, figsize=(15, 15))
arr[0].imshow(X[image_index])
arr[0].set_title('Processed Image')
arr[1].imshow(y[image_index])
arr[1].set_title('Processed Masked Image ')


In [ ]:
# doing the prediction
img = X[0]
img = img[np.newaxis, ...]
pred_y = unet.predict(img)
pred_mask_test = tf.argmax(pred_y[0], axis=-1)
pred_mask_test = pred_mask_test[..., tf.newaxis]

first = True
#for each box do the prediction and get the results
for i in range(1,X.shape[0]):
# doing the prediction
  img = X[i]
  img = img[np.newaxis, ...]
  pred_y = unet.predict(img)
  pred_mask = tf.argmax(pred_y[0], axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  
  if first:
    blocks = tf.stack((pred_mask_test[:,:,0], pred_mask[:,:,0]), axis=0)
    gt_blocks = tf.stack((y[0], y[i]), axis=0)
    first = False
  else:
    add = tf.concat(pred_mask[:,:,0], axis=1)
    add = tf.reshape(add, [1,target_shape_img_blok[0], target_shape_img_blok[1]])
    blocks = tf.concat((blocks, add), axis=0)

    add = tf.concat(y[i], axis=1)
    add = tf.reshape(add, [1,target_shape_img_blok[0], target_shape_img_blok[1]])
    gt_blocks = tf.concat((gt_blocks, add), axis=0)

In [ ]:
shape_images_dict.keys()

In [ ]:
#create the images
shape_block = target_shape_img_blok
image_shape_pad = [shape_images_dict["z802.png"], shape_images_dict["z822.png"]]
Test_images_approach_2 = "/content/drive/MyDrive/binarization_data/model_images/Test_images_approach_2"

#create a directory with the name of the validation images
dir_val_img = ["z802", "z822"]
file_name = "F1s.png" #name of the file im going to take to reverse the passing


b = 0
for i in range(2): #for each validation images
  img_shape = image_shape_pad[i]
  n_blocks_for_image = round((img_shape[0] * img_shape[1]) / (shape_block[0] * shape_block[1]))
  

  # reconstruct the predicted images, and save thos in the apposit directory
  tiles = tf.slice(blocks, begin=[b, 0, 0], size=[n_blocks_for_image, shape_block[0], shape_block[1]])
  img = unsplit_image(tiles, img_shape)
  

  #Crete the GT image
  gt_tiles = tf.slice(gt_blocks, begin=[b, 0, 0], size=[n_blocks_for_image, shape_block[0], shape_block[1]])
  gt_img = unsplit_image(gt_tiles, img_shape)

  b += n_blocks_for_image


  #remove the padding before saving the images

  #get the image without padding
  img_no_pad = imageio.imread(os.path.join(Test_images_approach_2, "val_images", dir_val_img[i], file_name))
  img = reverse_padding(img_no_pad, img)
  gt_img = reverse_padding(img_no_pad, gt_img)

  #save the image
  plt.imsave(f"{Test_images_approach_2}/output/gt/FN{i}_GT.png", gt_img)
  plt.imsave(f"{Test_images_approach_2}/output/images/FN{i}.png", img)

